## Environment Set Up

In [1]:
# Imports
import os
import pandas as pd
from pandas.io.json import json_normalize
import geopandas as gpd
from awhere_geolocation import AWhereAPI
import extract_data_geolocation
import clean_data_geolocation

In [2]:
# Show all Pandas dataframe columns
pd.set_option('display.max_columns', None)

In [3]:
# Define aWhere API key and secret
api_key = os.environ.get('AWHERE_API_KEY')
api_secret = os.environ.get('AWHERE_API_SECRET')

In [4]:
# Create aWhere object
aWhere = AWhereAPI(api_key, api_secret)

In [5]:
# Define coordinates
rmnp_latitude = 40
rmnp_longitude = -105

## Soil Data Combined (Temp, Moisture)

In [58]:
# Get forecast in raw form
forecast = aWhere.get_weather_forecast(
    rmnp_latitude, rmnp_longitude)

In [12]:
forecast

{'date': '2020-04-12',
 'location': {'latitude': 40.0, 'longitude': -105.0},
 'forecast': [{'startTime': '2020-04-12T00:00:00+00:00',
   'endTime': '2020-04-12T23:59:59+00:00',
   'conditionsCode': 'F31',
   'conditionsText': 'Cloudy, Moderate Rain, Light Wind/Calm',
   'temperatures': {'max': 18.91748046875,
    'min': -3.0465455055236816,
    'units': 'C'},
   'precipitation': {'chance': 100.0,
    'amount': 3.0624999962747097,
    'units': 'mm'},
   'sky': {'cloudCover': 100.0, 'sunshine': 0.0},
   'solar': {'amount': 5844.0, 'units': 'Wh/m^2'},
   'relativeHumidity': {'average': 53.0625,
    'max': 90.9000015258789,
    'min': 18.299999237060547},
   'wind': {'average': 5.921989223831704,
    'max': 10.043753588919282,
    'min': 2.6060634788570836,
    'units': 'm/sec',
    'bearing': 4.0,
    'direction': 'N'},
   'dewPoint': {'amount': -5.203474215183234, 'units': 'C'},
   'soilTemperatures': [{'depth': '0-0.1 m below ground',
     'average': 11.364468336105347,
     'max': 17.6

In [56]:
def extract_forecast_soil(forecast):
    """Extract aWhere forecast soil temperature
    data and returns it in a pandas dataframe.
    """
    # Initialize lists to store soil dataframes
    forecast_soil_list = []

    # Check if more than one day
    if forecast.get('forecasts'):
        forecast_iterator = json_normalize(forecast.get('forecasts'))

    # Single day
    else:
        forecast_iterator = json_normalize(forecast)

    # Loop through each row in the top-level flattened dataframe
    for index, row in forecast_iterator.iterrows():

        # Extract date, lat, lon for insertion into lower-level dataframe outputs
        date = row['date']
        lat = row['location.latitude']
        lon = row['location.longitude']

        # Get soil temperature data
        forecast_soil_temp = row['forecast'][0].get('soilTemperatures')
        forecast_soil_moisture = row['forecast'][0].get('soilMoisture')

        # Flatten data into dataframe
        forecast_soil_temp_df = json_normalize(forecast_soil_temp)
        forecast_soil_moisture_df = json_normalize(forecast_soil_moisture)

        # Combine temperature and moisture
        forecast_soil_df = forecast_soil_temp_df.merge(
            forecast_soil_moisture_df, on='depth', suffixes=('_temp', '_moisture'))

        # Assign date, lat/lon to dataframe
        forecast_soil_df['date'] = date
        forecast_soil_df['latitude'] = lat
        forecast_soil_df['longitude'] = lon

        # Shorten depth values to numerics (will be used in MultiIndex)
        forecast_soil_df['depth'] = forecast_soil_df['depth'].apply(lambda x: x[0:-15])

        # Rename depth prior to indexing
        forecast_soil_df.rename(columns={'depth': 'ground_depth_m'}, inplace=True)

        # Create multi-index dataframe for date and soil depth (rename depth columns? rather long)
        soil_multi_index = forecast_soil_df.set_index(
            ['date', 'ground_depth_m'])

        # Add dataframe to list of dataframes
        forecast_soil_list.append(soil_multi_index)
    
    # Return merged lists of dataframes into a single dataframe
    return pd.concat(forecast_soil_list)

In [59]:
extract_forecast_soil(forecast)

average_temp   max_temp   min_temp units  \
date       ground_depth_m                                             
2020-04-11 0-0.1              13.616608  18.091724  10.198389     C   
           0.1-0.4            10.167496  10.330000  10.090000     C   
           0.4-1               8.070495   8.172949   7.972969     C   
           1-2                 7.614110   7.621758   7.601758     C   
2020-04-12 0-0.1              11.364468  17.637533   7.704883     C   
           0.1-0.4            10.356250  10.530000  10.060000     C   
           0.4-1               8.301961   8.392364   8.186081     C   
           1-2                 7.640649   7.651758   7.623314     C   
2020-04-13 0-0.1               3.168801   7.906201   1.050000     C   
           0.1-0.4             9.529854  10.023334   8.981797     C   
           0.4-1               8.449098   8.472363   8.402357     C   
           1-2                 7.664674   7.671758   7.651758     C   
2020-04-14 0-0.1               0.899227   4.335864  -0.849951     C   
           0.1-0.4             8.382195   8.935319   7.893535     C   
           0.4-1               8.454861   8.472363   8.412363     C   
           1-2                 7.688257   7.701172   7.670000     C   
2020-04-15 0-0.1               3.952691   8.168286   2.213550     C   
           0.1-0.4             7.582927   7.870397   7.408340     C   
           0.4-1               8.360967   8.412168   8.292363     C   
           1-2                 7.707567   7.711758   7.701367     C   
2020-04-16 0-0.1               6.669709  11.850000   3.662988     C   
           0.1-0.4             7.376755   7.476250   7.246250     C   
           0.4-1               8.238119   8.289030   8.182364     C   
           1-2                 7.721540   7.731172   7.711758     C   
2020-04-17 0-0.1               7.261854  11.411432   5.449975     C   
           0.1-0.4             7.576578   7.654160   7.492090     C   
           0.4-1               8.150197   8.182364   8.112363     C   
           1-2                 7.728874   7.731777   7.721172     C   
2020-04-18 0-0.1               6.073760  11.577637   4.250000     C   
           0.1-0.4             7.342253   7.488757   7.202090     C   
           0.4-1               8.061366   8.108829   8.012246     C   
           1-2                 7.737611   7.741777   7.731777     C   
2020-04-19 0-0.1              10.798389  16.797680   7.063574     C   
           0.1-0.4             7.717637   8.168340   7.447494     C   
           0.4-1               7.998094   8.012363   7.991777     C   
           1-2                 7.748659   7.751777   7.741777     C   
2020-04-20 0-0.1              12.179790  16.146915   9.624487     C   
           0.1-0.4             8.570859   8.840429   8.231673     C   
           0.4-1               8.026752   8.062226   7.995690     C   
           1-2                 7.758255   7.761172   7.751172     C   

                           average_moisture  max_moisture  min_moisture  \
date       ground_depth_m                                                 
2020-04-11 0-0.1                  15.100000     15.299999     14.700000   
           0.1-0.4                18.349405     18.411905     18.111906   
           0.4-1                   5.903708      5.903708      5.903708   
           1-2                    13.943129     13.943129     13.943129   
2020-04-12 0-0.1                  15.091666     15.500000     14.599999   
           0.1-0.4                18.111906     18.111906     18.111906   
           0.4-1                   5.903708      5.903708      5.903708   
           1-2                    13.943129     13.943129     13.943129   
2020-04-13 0-0.1                  16.125000     18.199999     15.500000   
           0.1-0.4                18.078573     18.111906     18.011906   
           0.4-1                   5.903708      5.903708      5.903708   
           1-2                    13.943129     13.943129     13.9431

In [47]:
soil_multi_index

average_temp   max_temp   min_temp units  \
date       ground_depth_m                                             
2020-04-12 0-0.1              11.364468  17.637533   7.704883     C   
           0.1-0.4            10.356250  10.530000  10.060000     C   
           0.4-1               8.301961   8.392364   8.186081     C   
           1-2                 7.640649   7.651758   7.623314     C   

                           average_moisture  max_moisture  min_moisture  \
date       ground_depth_m                                                 
2020-04-12 0-0.1                  15.091666     15.500000     14.599999   
           0.1-0.4                18.111906     18.111906     18.111906   
           0.4-1                   5.903708      5.903708      5.903708   
           1-2                    13.943129     13.943129     13.943129   

                           latitude  longitude  
date       ground_depth_m                       
2020-04-12 0-0.1               40.0     -105.0  
           0.1-0.4             40.0     -105.0  
           0.4-1               40.0     -105.0  
           1-2                 40.0     -105.0

In [20]:
forecast_soil_moisture_df

,depth,average,max,min
0,0-0.1 m below ground,15.091666,15.500000,14.599999
1,0.1-0.4 m below ground,18.111906,18.111906,18.111906
2,0.4-1 m below ground,5.903708,5.903708,5.903708
3,1-2 m below ground,13.943129,13.943129,13.943129


In [28]:
soil_data = forecast_soil_temp_df.merge(forecast_soil_moisture_df, on='depth',
    suffixes=('_temp', '_moisture'))

In [29]:
soil_data

,depth,average_temp,max_temp,min_temp,units,average_moisture,max_moisture,min_moisture
0,0-0.1 m below ground,11.364468,17.637533,7.704883,C,15.091666,15.500000,14.599999
1,0.1-0.4 m below ground,10.356250,10.530000,10.060000,C,18.111906,18.111906,18.111906
2,0.4-1 m below ground,8.301961,8.392364,8.186081,C,5.903708,5.903708,5.903708
3,1-2 m below ground,7.640649,7.651758,7.623314,C,13.943129,13.943129,13.943129


In [30]:
forecast_soil_moisture_df

,depth,average,max,min
0,0-0.1 m below ground,15.091666,15.500000,14.599999
1,0.1-0.4 m below ground,18.111906,18.111906,18.111906
2,0.4-1 m below ground,5.903708,5.903708,5.903708
3,1-2 m below ground,13.943129,13.943129,13.943129


In [31]:
forecast_soil_temp_df

,depth,average,max,min,units
0,0-0.1 m below ground,11.364468,17.637533,7.704883,C
1,0.1-0.4 m below ground,10.356250,10.530000,10.060000,C
2,0.4-1 m below ground,8.301961,8.392364,8.186081,C
3,1-2 m below ground,7.640649,7.651758,7.623314,C
